In [31]:
import time
import numpy as np
import torch.nn as nn
import torch
import struct
from sklearn.metrics import accuracy_score

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [33]:
class KNNClassifier:
        
    def __init__(self, k=3, metric=None):
        self.k = k
        if metric is None:
            metric = 'euclidean'
        self.metric = metric

    def fit(self, X_train: torch.Tensor, y_train: torch.Tensor):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test: torch.Tensor):
        y_pred = self._cal_predictions(X_test)
        return y_pred

    def _cal_predictions(self, X: torch.Tensor):
        idx = None
        dist = torch.cdist(self.X_train, X, p=2)
        idx =  torch.argsort(dist, dim=0)[:self.k]
        # pick the most frequently occuring label from top k neighbors 
        k_labels = self.y_train[idx]
        return torch.mode(k_labels, dim=0)[0]


In [34]:
# reading mnist byte
# Reference "https://stackoverflow.com/questions/39969045/parsing-yann-lecuns-mnist-idx-file-format"
def read_mnist_idx_images(path):
    with open(path, 'rb') as f:
        magic, size = struct.unpack(">II", f.read(8))
        nrows, ncols = struct.unpack(">II", f.read(8))
        imgs = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
        imgs = imgs.reshape((size, nrows, ncols))
    return imgs


def read_mnist_idx_labels(path):
    with open(path, 'rb') as f:
        magic, size = struct.unpack(">II", f.read(8))
        labels = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
        labels = labels.reshape((size,))
    return labels

In [35]:
mnist_train, mnist_test = dict(), dict()
mnist_train['X'] = read_mnist_idx_images('./data/MNIST/train-images.idx3-ubyte')
mnist_train['y'] = read_mnist_idx_labels('./data/MNIST/train-labels.idx1-ubyte')

mnist_test['X'] = read_mnist_idx_images('./data/MNIST/t10k-images.idx3-ubyte')
mnist_test['y'] = read_mnist_idx_labels('./data/MNIST/t10k-labels.idx1-ubyte')

In [36]:
def zscore_norm(data: np.ndarray) -> np.ndarray:
    mean = np.mean(data, axis=1).reshape(data.shape[0], 1)
    std = np.std(data, axis=1).reshape(data.shape[0], 1)
    z_normed = (data - mean) / std
    return np.asarray(z_normed, dtype=np.float16)

In [37]:
mnist_tr_10k_X = mnist_train['X'][:9000]
mnist_tr_y = mnist_train['y'][:9000]

mnist_ts_10k_X = mnist_test['X'][:5000]
mnist_ts_y = mnist_test['y'][:5000]

# train
mnist_tr_X = zscore_norm(mnist_tr_10k_X.reshape(mnist_tr_10k_X.shape[0], 784))
# test
mnist_ts_X = zscore_norm(mnist_ts_10k_X.reshape(mnist_ts_10k_X.shape[0], 784))

In [38]:
# GPU
knn = KNNClassifier(k=5, metric='euclidean')
knn.fit(torch.from_numpy(mnist_tr_X).cuda(), torch.from_numpy(mnist_tr_y).cuda())
st_time = time.time()
y_pred = knn.predict(torch.from_numpy(mnist_ts_X).cuda())
ed_time = time.time()
print(f'Accuracy: {accuracy_score(mnist_ts_y,
                                   y_pred.detach().cpu().numpy())}')
print(f'Execution time: {ed_time - st_time} s')

Accuracy: 0.9342
Execution time: 0.00592350959777832 s


In [39]:
torch.cuda.empty_cache()

In [40]:
# CPU
knn = KNNClassifier(k=5, metric='euclidean')
knn.fit(torch.from_numpy(mnist_tr_X), torch.from_numpy(mnist_tr_y))
st_time = time.time()
y_pred = knn.predict(torch.from_numpy(mnist_ts_X))
ed_time = time.time()
print(f'Accuracy: {accuracy_score(mnist_ts_y,
                                   y_pred.detach().cpu().numpy())}')
print(f'Execution time: {ed_time - st_time} s')

Accuracy: 0.9348
Execution time: 94.10724544525146 s


cdist and argsort run 19000x times faster on GPU